In [5]:
import sys
import pathlib
import numpy as np
import os
import numpy as np
import os
import subprocess

path_to_deep_mod = '/mnt/mbi/home/e0031794/Documents/DeePyMoD/src'
sys.path.append(path_to_deep_mod)

from deepymod.DeepMoD import DeepMoD
from deepymod.library_functions import library_1D
from deepymod.utilities import print_PDE
from deepymod.utilities import library_matrix_mat

class GPUGetter:

    def get_free_gpu(self):

        """
            Selects the gpu with the most free memory

        """
        output = subprocess.Popen('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free', stdout=subprocess.PIPE,
                              shell=True).communicate()[0]
        output = output.decode("ascii")

        # assumes that it is on the popiah server and the last gpu is not used
        memory_available = [int(x.split()[2]) for x in output.split("\n")[:-2]]

        if memory_available:
            print("Setting GPU to use to PID {}".format(np.argmax(memory_available)))
            return np.argmax(memory_available)

        if not memory_available:
            print('No GPU memory available')

class GPUSetter:

    def set_max_gpu(self):

        """
            Sets GPU with maximum memory

        """
        gpu_getter = GPUGetter()
        gpu = str(gpu_getter.get_free_gpu())

        if gpu:
            print("Using GPU: %s" % gpu)
            os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
            os.environ['CUDA_VISIBLE_DEVICES'] = gpu

        if not gpu:
            print('No GPU detected')
class NormalTerm:

    """
        Stores non-derivative PDE terms

    """
    def __init__(self):
        self.u = ['1', 'u', 'uˆ2']
    
    def get_normal_terms(self):
        return self.u

class DerivativeTerm:

    """
        Stores non-derivative PDE terms

    """
    def __init__(self):
        self.du = ['1', 'u_{x}', 'u_{xx}', 'u_{xxx}']
    
    def get_derivative_terms(self):
        return self.du

class LibraryTerms:

    def __init__(self):
        self.normal_term = NormalTerm()
        self.derivative_term = DerivativeTerm()

    def get_library(self):

        """
            Gets a list of pde terms generated using cartersian product from terms originating from NormalTerm and DerivativeTerm.
            The list of pde terms is a library of basis terms.

        """

        normal_terms = self.normal_term.get_normal_terms()
        derivative_terms = self.derivative_term.get_derivative_terms()

        library = library_matrix_mat(normal_terms, derivative_terms)

        return library
    
    def get_library_len(self):
        library = self.get_library()
        return len(library)

class LibraryConfig:

    def get_library_config(self):

        """
            Gets configuration of pde library 
        """

        num_library_terms = LibraryTerms().get_library_len()

        library_config = {'total_terms': num_library_terms, 'deriv_order': 3, 'poly_order': 2}

        return library_config

class PDEManager:

    def get_lib_config_and_terms(self):
        lib_config = LibraryConfig().get_library_config()
        library = LibraryTerms().get_library()

        return lib_config, library
    
class LayerParameter:

    """
        Stores parameters for DeepMod's architecture
        
    """

    def __init__(self):
        self.layers = {'layers': [2, 20, 20, 20, 20, 20, 1], 'lambda': 10e-6}
    
    def get_layers(self):
        return self.layers

class TrainingParameter:

    """
        Stores parameters for DeepMod's training

    """
    def __init__(self):
        self.training_param = {'max_iterations': 50000, 'grad_tol': 10**-6, 'learning_rate': 0.002, 'beta1': 0.99, 
        'beta2': 0.999, 'epsilon': 10 ** -8}
    
    def get_trng_config(self):
        return self.training_param

class ParamGetter:

    def get_trng_layer_params(self):
        return TrainingParameter().get_trng_config(), LayerParameter().get_layers()

class PdePrinter:

    def save_pde(self, sparse_vector, lib_terms, save_file):
        """
            sparse_vector (np array): sparse vector which is output from deepmod
            lib_terms (list): libary of pde basis terms 
            save_file (string): full path of file which a pde equation will be printed to

        """
        result = print_PDE(sparse_vector[0], lib_terms, PDE_term="u_t")
        with open(save_file, 'w+', encoding='utf-8') as f:
            f.write(result)

def pipeline(data_dir):
    #data_dir is an absolute path
    all_files_folders = os.listdir(data_dir)
    target_folders = [os.path.join(data_dir, obj) for obj in all_files_folders if '_subset_scaled' in obj] #search for subfolders

    #get all necessary configs
    lib_config, libary_terms = PDEManager().get_lib_config_and_terms()
    training_config, nn_layer_config = ParamGetter().get_trng_layer_params()
    output_config = {"output_directory": '', 'X_predict': ''}

    GPUSetter().set_max_gpu()

    #subset == amt of subset from original data. i.e amt of slicing done
    for folder in target_folders:

        if '.npy' in folder or '.txt' in folder:
            continue

        subset = folder.split('/')[-1].split('_')[0]  #get subset i.e 800, 900
        
        if subset != '9700':
            continue
        print('processing folder: ', folder)

        for_google_dir = os.path.join(folder + '/google_drive_storage') #we want to store relevant outputs in a folder and upload into google drive
        pathlib.Path(for_google_dir).mkdir(parents=True,exist_ok=True)
        
        full_x_t = np.load(os.path.join(folder, 'space_time_data_full_' + subset + '.npy'))
        x_t_train = np.load(os.path.join(folder, 'space_time_data_sampled_' + subset + '.npy'))
        bicoid_train = np.load(os.path.join(folder, 'bicoid_sampled_' + subset + '.npy'))
        
        save_dir =  os.path.join(folder, 'Out_subset_' + subset + '_expt')
        output_config['output_directory'] = save_dir
        output_config['X_predict'] = full_x_t

        sparse_vector, denoised = DeepMoD(
            x_t_train, bicoid_train, nn_layer_config,
            library_1D, lib_config, training_config, 
            output_config
            )

        #save results
        save_file_path = os.path.join(for_google_dir, 'pde_result_subset_' + subset + '.txt')
        PdePrinter().save_pde(sparse_vector, libary_terms, save_file_path)

        sparse_path = os.path.join(for_google_dir, 'sparse_vec_subset' + subset)
        denoised_path = os.path.join(for_google_dir, 'denoised_result_subset' + subset)

        #save training output and plots
        np.save(sparse_path, sparse_vector)
        np.save(denoised_path, denoised)

if __name__ == "__main__":
    data_dir = '/mnt/mbi/home/e0031794/Documents/FYP/FYP_results_11_9_2019/data_slicing_DD_3/1_trial'
    pipeline(data_dir)

Setting GPU to use to PID 1
Using GPU: 1
processing folder:  /mnt/mbi/home/e0031794/Documents/FYP/FYP_results_11_9_2019/data_slicing_DD_3/1_trial/9700_subset_scaled
Epoch | Total loss | Loss gradient | MSE | PI | L1 
0 [0.87129223, 0.010348889, 0.48670176, 0.38417163, 0.00041884967]
500 [0.07393331, 0.0007590751, 0.07167128, 0.0012707759, 0.0009912567]
1000 [0.0721736, 0.00012455664, 0.07141145, 4.54711e-05, 0.0007166814]
1500 [0.0685244, 0.0006093475, 0.06833519, 3.0002171e-05, 0.00015920386]
2000 [0.035761964, 0.02254741, 0.034927018, 0.0008174137, 1.7530901e-05]
2500 [0.0054898756, 0.03019267, 0.005074015, 0.0004037214, 1.2139148e-05]
3000 [0.0020884878, 0.01937469, 0.0019452568, 0.00013218721, 1.1043947e-05]
3500 [0.0012405072, 0.061620094, 0.001032081, 0.00019772098, 1.0705374e-05]
4000 [0.000861329, 0.037358385, 0.00075242535, 9.872576e-05, 1.017792e-05]
4500 [0.000687894, 0.013914771, 0.0006323825, 4.5895045e-05, 9.616424e-06]
5000 [0.00060679205, 0.00627141, 0.00056361227, 3.37

KeyboardInterrupt: 

In [7]:
tensorboard

NameError: name 'tensorboard' is not defined